# RL in Finance(Test Cash Penalty) 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sunnyswag/RL_in_Finance/blob/main/RL_in_Finance_Test_cash_penalty.ipynb)

## 1、拉取 github 仓库，下载并导入相关包
&emsp;&emsp;运行流程：python setup.py -> pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
import time

%matplotlib inline
from utils import config
from utils.pull_data import Pull_data
from utils.preprocessors import FeatureEngineer, split_data
from utils.env_retreat_penalty import StockTradingEnvRetreatpenalty
from utils.models import DRL_Agent
from utils.backtest import backtest_stats, backtest_plot, get_baseline
import itertools
import sys
import codecs
sys.path.append("../RL_in_Finance")
# sys.stdout = codecs.getwriter("utf-8")(sys.stdout.detach())

## 2、下载数据

数据来源：Tushare API<br>
当前用到的数据：SSE_50 和 CSI_300<br>
数据量的大小：shape[2892 * n, 8]

In [2]:
# stock_list = config.SSE_50
# df = Pull_data(stock_list, save_data=False).pull_data()

--- 开始下载 ----
下载进度 : 20.0%
下载进度 : 40.0%
下载进度 : 60.0%
下载进度 : 80.0%
下载进度 : 100.0%
--- 下载完成 ----
DataFrame 的大小:  (114845, 8)


In [3]:
# df.sort_values(['date', 'tic'], ignore_index=True).head()

,date,tic,open,high,low,close,volume,day
0,2009-01-05,600000.SH,2.7584,2.8115,2.7258,2.8013,503142.56,0
1,2009-01-05,600009.SH,9.4665,9.6505,9.4414,9.5836,52100.33,0
2,2009-01-05,600016.SH,1.6369,1.6487,1.6170,1.6448,947739.82,0
3,2009-01-05,600028.SH,3.1966,3.2461,3.1786,3.2371,361085.66,0
4,2009-01-05,600030.SH,9.2667,9.5705,9.1654,9.5502,818978.41,0


In [4]:
# print("数据下载的时间区间为：{} 至 {}".format(config.Start_Date, config.End_Date))

数据下载的时间区间为：20090101 至 20210101


In [5]:
# print("下载的股票列表为: ")
# print(stock_list)

下载的股票列表为: 
['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', '600036.SH', '600048.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH', '600585.SH', '600588.SH', '600690.SH', '600703.SH', '600745.SH', '600837.SH', '600887.SH', '600918.SH', '601012.SH', '601066.SH', '601088.SH', '601138.SH', '601166.SH', '601186.SH', '601211.SH', '601236.SH', '601288.SH', '601318.SH', '601319.SH', '601336.SH', '601398.SH', '601601.SH', '601628.SH', '601668.SH', '601688.SH', '601816.SH', '601818.SH', '601857.SH', '601888.SH', '603160.SH', '603259.SH', '603288.SH', '603501.SH', '603986.SH']


## 3、数据预处理

In [6]:
# processed_df = FeatureEngineer(use_technical_indicator=True).preprocess_data(df)
# processed_df['log_volume'] = np.log(processed_df.volume*processed_df.close)
# processed_df['change'] = (processed_df.close-processed_df.open)/processed_df.close
# processed_df['daily_variance'] = (processed_df.high-processed_df.low)/processed_df.close

成功添加技术指标
对当前时间段未上市的公司的所有行置零


/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
# processed_df = processed_df.fillna(0)

In [8]:
# print("技术指标列表: ")
# print(config.TECHNICAL_INDICATORS_LIST)
# print("技术指标数: {}个".format(len(config.TECHNICAL_INDICATORS_LIST)))

技术指标列表: 
['boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'macd', 'volume_20_sma', 'volume_60_sma', 'volume_120_sma', 'close_20_sma', 'close_60_sma', 'close_120_sma']
技术指标数: 12个


In [9]:
# processed_df.head()

,date,tic,open,high,low,close,volume,day,boll_ub,boll_lb,...,macd,volume_20_sma,volume_60_sma,volume_120_sma,close_20_sma,close_60_sma,close_120_sma,log_volume,change,daily_variance
0,2009-01-05,600000.SH,2.7584,2.8115,2.7258,2.8013,503142.56,0.0,0.0,0.0,...,0.0,503142.56,503142.56,503142.56,2.8013,2.8013,2.8013,14.158712,0.015314,0.030593
1,2009-01-05,600009.SH,9.4665,9.6505,9.4414,9.5836,52100.33,0.0,0.0,0.0,...,0.0,52100.33,52100.33,52100.33,9.5836,9.5836,9.5836,13.120980,0.012219,0.021819
2,2009-01-05,600016.SH,1.6369,1.6487,1.6170,1.6448,947739.82,0.0,0.0,0.0,...,0.0,947739.82,947739.82,947739.82,1.6448,1.6448,1.6448,14.259454,0.004803,0.019273
3,2009-01-05,600028.SH,3.1966,3.2461,3.1786,3.2371,361085.66,0.0,0.0,0.0,...,0.0,361085.66,361085.66,361085.66,3.2371,3.2371,3.2371,13.971548,0.012511,0.020852
4,2009-01-05,600030.SH,9.2667,9.5705,9.1654,9.5502,818978.41,0.0,0.0,0.0,...,0.0,818978.41,818978.41,818978.41,9.5502,9.5502,9.5502,15.872375,0.029685,0.042418


In [3]:
# processed_df.to_csv("processed_df.csv", index = False)
processed_df = pd.read_csv("processed_df.csv")

In [4]:
train_data = split_data(processed_df, config.Start_Trade_Date, config.End_Trade_Date)
test_data = split_data(processed_df, config.End_Trade_Date, config.End_Test_Date)

In [5]:
print("训练数据的范围：{} 至 {}".format(config.Start_Trade_Date, config.End_Trade_Date))
print("测试数据的范围：{} 至 {}".format(config.End_Trade_Date, config.End_Test_Date))
print("训练数据的长度: {},测试数据的长度:{}".format(len(train_data), len(test_data)))
print("训练集数据 : 测试集数据: {} : {}".format(round(len(train_data)/len(test_data),1), 1))

训练数据的范围：2009-01-01 至 2019-01-01
测试数据的范围：2019-01-01 至 2021-01-01
训练数据的长度: 121550,测试数据的长度:24350
训练集数据 : 测试集数据: 5.0 : 1


In [6]:
train_data.head()

,date,tic,open,high,low,close,volume,day,boll_ub,boll_lb,...,macd,volume_20_sma,volume_60_sma,volume_120_sma,close_20_sma,close_60_sma,close_120_sma,log_volume,change,daily_variance
0,2009-01-05,600000.SH,2.7584,2.8115,2.7258,2.8013,503142.56,0.0,0.0,0.0,...,0.0,503142.56,503142.56,503142.56,2.8013,2.8013,2.8013,14.158712,0.015314,0.030593
0,2009-01-05,600009.SH,9.4665,9.6505,9.4414,9.5836,52100.33,0.0,0.0,0.0,...,0.0,52100.33,52100.33,52100.33,9.5836,9.5836,9.5836,13.120980,0.012219,0.021819
0,2009-01-05,600016.SH,1.6369,1.6487,1.6170,1.6448,947739.82,0.0,0.0,0.0,...,0.0,947739.82,947739.82,947739.82,1.6448,1.6448,1.6448,14.259454,0.004803,0.019273
0,2009-01-05,600028.SH,3.1966,3.2461,3.1786,3.2371,361085.66,0.0,0.0,0.0,...,0.0,361085.66,361085.66,361085.66,3.2371,3.2371,3.2371,13.971548,0.012511,0.020852
0,2009-01-05,600030.SH,9.2667,9.5705,9.1654,9.5502,818978.41,0.0,0.0,0.0,...,0.0,818978.41,818978.41,818978.41,9.5502,9.5502,9.5502,15.872375,0.029685,0.042418


In [7]:
test_data.head()

,date,tic,open,high,low,close,volume,day,boll_ub,boll_lb,...,macd,volume_20_sma,volume_60_sma,volume_120_sma,close_20_sma,close_60_sma,close_120_sma,log_volume,change,daily_variance
0,2019-01-02,600000.SH,8.9589,9.0049,8.8117,8.9221,237628.22,2.0,10.502893,8.657577,...,-0.251945,2.152395e+05,2.430301e+05,2.210339e+05,9.580235,9.716063,9.475849,14.566994,-0.004125,0.021654
0,2019-01-02,600009.SH,49.6941,49.8119,48.8298,49.4780,40051.97,2.0,51.140976,48.044854,...,0.025528,3.263298e+04,5.353243e+04,5.799566e+04,49.592915,49.906370,53.102410,14.499461,-0.004368,0.019849
0,2019-01-02,600016.SH,5.0982,5.1071,5.0002,5.0358,579415.00,2.0,5.554261,4.922929,...,-0.108006,6.324888e+05,6.533327e+05,6.093035e+05,5.238595,5.419815,5.361655,14.886347,-0.012391,0.021228
0,2019-01-02,600028.SH,4.4596,4.4684,4.3372,4.3809,2185583.70,2.0,5.502448,4.478342,...,-0.185334,1.141945e+06,1.169240e+06,1.067717e+06,4.990395,5.276185,5.487852,16.074648,-0.017964,0.029948
0,2019-01-02,600030.SH,0.0000,0.0000,0.0000,0.0000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,-inf,0.000000,0.000000


## 4、初始化环境

**state_space 由四部分组成 :** <br>
1. 当天的资金量
2. 每只股票当天的收盘价
3. 每只股票当天的持仓量
4. 股票数 * 技术指标数<br>
5. 当天成交量

**reward 的计算方式：**<br>
* reward 交易前的总资产-当天交易后的总资产 = 当天交易的手续费
* TODO：待改进

**action_space 的空间：**<br>
  * actions ∈[-100, 100]
  * 正数表示买入，负数表示卖出，0表示不进行买入卖出操作
  * 绝对值表示买入卖出的数量

In [8]:
# stock_dimension = len(df.tic.unique())
# state_space = 1 + 2*stock_dimension + \
#     len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension + stock_dimension
# print("stock_dimension: {}, state_space: {}".format(stock_dimension, state_space))

In [ ]:
# 初始化环境的参数
information_cols = config.TECHNICAL_INDICATORS_LIST + ["close", "day", "log_volume", "change", "daily_variance"]

e_train_gym = StockTradingEnvRetreatpenalty(df = train_data,initial_amount = 1e6,hmax = 5000, 
                                turbulence_threshold = None, 
                                currency='￥',
                                buy_cost_pct=3e-3,
                                sell_cost_pct=3e-3,
                                cache_indicator_data=True,
                                daily_information_cols = information_cols, 
                                print_verbosity = 500,
                                patient=True,
                                random_start = True)

e_trade_gym = StockTradingEnvRetreatpenalty(df = test_data,initial_amount = 1e6,hmax = 5000, 
                                turbulence_threshold = None, 
                                currency='￥',
                                buy_cost_pct=3e-3,
                                sell_cost_pct=3e-3,
                                cache_indicator_data=True,
                                daily_information_cols = information_cols, 
                                print_verbosity = 500,
                                patient=True,
                                random_start = False)

caching data


In [9]:
# 对环境进行测试
# %debug
observation = e_train_gym.reset() # 初始化环境，observation为环境状态
count = 0
total_reward = 0
for t in range(300):
    action = e_train_gym.action_space.sample() # 随机采样动作
    observation, reward, done, info = e_train_gym.step(action) # 与环境交互，获得下一个state的值
    total_reward += reward
    if done:             
        break
    count+=1
    # time.sleep(0.2)      #每次等待 0.2s
print("count: ", count)
print("reward: {}, done: {}".format(total_reward, done))

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|RETREAT_PROPORTION


/home/RL_in_Finance/utils/env_retreat_penalty.py:257: RuntimeWarning: invalid value encountered in true_divide
  actions = actions / self.closings


   0| 173|CASH SHORTAGE  |￥10,960        |￥871,500       |-26.07715%|-12.85003%|-13.23%   
   0| 179|CASH SHORTAGE  |￥8,856         |￥915,643       |-17.26757%|-8.43569% |-8.83%    
   0| 199|CASH SHORTAGE  |￥1,035         |￥1,035,923     |3.59233%  |3.59233%  |0.00%     
   0| 210|CASH SHORTAGE  |￥9,775         |￥1,088,865     |7.53310%  |8.88647%  |-1.35%    
   0| 237|CASH SHORTAGE  |￥37,052        |￥953,038       |-20.53233%|-4.69621% |-15.84%   
   0| 247|CASH SHORTAGE  |￥1,372         |￥1,044,549     |-3.29970% |4.45494%  |-7.75%    
   0| 253|CASH SHORTAGE  |￥26,408        |￥1,015,813     |-8.71100% |1.58135%  |-10.29%   
   0| 286|CASH SHORTAGE  |￥6,633         |￥987,083       |-14.12127%|-1.29170% |-12.83%   
count:  300
reward: -36.66621777824988, done: False


In [ ]:
import multiprocessing

n_cores = multiprocessing.cpu_count() - 10
n_cores = 24
print("using {} cores".format(n_cores))

#this is our training env. It allows multiprocessing
env_train, _ = e_train_gym.get_multiproc_env(n = n_cores)

#this is our observation environment. It allows full diagnostics
env_trade, _ = e_trade_gym.get_sb_env()

## 5、开始训练

所用到的框架：stable_baseline3

In [20]:
agent = DRL_Agent(env = env_train)

In [21]:
# from torch.nn import Softsign, ReLU
ppo_params ={'n_steps': 256, 
             'ent_coef': 0.0, 
             'learning_rate': 0.000005, 
             'batch_size': 256, 
            'gamma': 0.99}

policy_kwargs = {
#     "activation_fn": ReLU,
    "net_arch": [1024 for _ in range(10)], 
#     "squash_output": True
}

model = agent.get_model("ppo",  
                        model_kwargs = ppo_params, 
                        policy_kwargs = policy_kwargs, verbose = 0)

# model = model.load("scaling_reward.model", env = env_train)

{'n_steps': 256, 'ent_coef': 0.0, 'learning_rate': 5e-06, 'batch_size': 256, 'gamma': 0.99}


/opt/conda/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370116979/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
model.learn(total_timesteps = 30000, 
            eval_env = env_trade, 
            eval_freq = 500,
            log_interval = 1, 
            tb_log_name = 'env_cashpenalty_highlr',
            n_eval_episodes = 1)   

/opt/conda/lib/python3.6/site-packages/stable_baselines3/common/callbacks.py:327: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x7f091aa59b70> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f08e3a91320>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")
/opt/conda/lib/python3.6/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/site-packages/tensorboard/summary/writer/event_file_writer.py", line 238, in run
    self._record_writer.write(data)
  File "/opt/conda/lib/python3.6/site-packages/tensorboard/summary/writer/record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 531, in write
    self.fs.append(self.filename, file_content, self.binary_mode)
  File "/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 154, in append
    self._write(filename, file_content, "ab" if binary_mode else "a")
  File "/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 158, in _write
    with io.open(filename, mode, e

In [ ]:
model.save("scaling_reward_{}_cores.model".format(n_cores))

## 6、测试

In [ ]:
df_account_value, df_actions = DRL_prediction(
    model=model, 
    environment = e_trade_gym)

In [ ]:
print("回测的时间窗口：{} 至 {}".format(config.End_Trade_Date, config.End_Test_Date))

回测的时间窗口：2019-01-01 至 2021-01-01


In [ ]:
df_account_value.to_csv("df_account_value.csv", index=False)
print("查看日账户净值")
print("开始: ")
print(df_account_value.head())
print("")
print("结束: ")
print(df_account_value.tail())

In [ ]:
print("查看每日所作的交易")
df_actions.to_csv("df_actions.csv", index=False)
df_actions.tail()

## 7、回测

In [ ]:
print("---------------------获取回测结果---------------------")
pref_stats_all = backtest_stats(account_value=df_account_value)

# perf_stats_all = pd.DataFrame(perf_stats_all)
# now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')
# perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
# 获取 baseline 的结果
print("---------------------获取baseline结果---------------------")
baseline_df = get_baseline(config.SSE_50_INDEX, 
              start="20190101",
              end="20210101")
baseline_stats = backtest_stats(baseline_df, value_col_name='close')

In [ ]:
# 删除 df_account_value 中重复的行
df_account_value.drop(df_account_value.index[1], inplace=True)

In [ ]:
baseline_df.head(10)

In [ ]:
print("---------------------Plot---------------------")
print("和 {} 指数进行比较".format(config.SSE_50_INDEX[0]))
%matplotlib inline
backtest_plot(df_account_value,
        baseline_start="20190101",
        baseline_end="20210101",
        baseline_ticker=config.SSE_50_INDEX,
      )